In [30]:
import pandas as pd
import numpy as np
from scipy.stats import norm, multivariate_normal, invwishart, invgamma
from numpy.linalg import inv

In [31]:
nsim_data = 10
J = 4
K = 2
zz = multivariate_normal.rvs(mean = np.zeros(K), size=nsim_data)
ww = np.array([[1, 3], [-2, 2],[-1,-1],[2,1]])
sigma = np.array([1,2,.3,.8])
Sigma = np.diag(sigma)
ee = multivariate_normal.rvs(mean = np.zeros(J), cov = Sigma, size=nsim_data)
yy = zz@ww.T + ee

In [128]:
data = dict()
data['N'] = nsim_data
data['K'] = K
data['J'] = J
data['sigma'] = sigma
data['Sigma'] = Sigma
data['y'] = yy
data['z'] = zz
data['w'] = ww
data['e'] = ee

In [33]:
print("dim(w) = ", ww.shape)
print("dim(z) = ", zz.shape)
print("dim(y) = ", yy.shape)
print("dim(e) = ", ee.shape)


dim(w) =  (4, 2)
dim(z) =  (10, 2)
dim(y) =  (10, 4)
dim(e) =  (10, 4)


In [51]:
# sample z
z_s = np.empty_like(zz)
for t in range(data['N']):
    inv_Sigma = inv(Sigma)
    cov1 = inv(np.eye(K) +  ww.T@inv_Sigma@ww)
    mean = cov1 @ ww.T@inv_Sigma@yy[t]
    z_s[t] = multivariate_normal.rvs(mean, cov1)


In [57]:
ww

array([[ 1,  3],
       [-2,  2],
       [-1, -1],
       [ 2,  1]])

In [69]:
i=0
ww[i,:(i+1)].reshape(1,1)

array([[1]])

In [121]:
def create_w_columns(i,k,ww):
    if i+1<=k:
        a = np.zeros(k)
        a[:(i+1)] = ww[i,:(i+1)]
        return a.T
    else:
        return ww[i,:]
        
create_w_columns(1,2,ww)

array([-2.,  2.])

In [122]:
create_w_columns(i,k=2, ww=ww).T.shape

(2,)

In [132]:
sigma_s = np.empty_like(sigma)
for i in range(data['J']):
    a0 = 1
    b0 = 1
    a = (data['N'] + a0 )/2.
    aux = yy[:,i] - zz@create_w_columns(i,k=2, ww=ww).T
    d = aux.T @ aux
    b = (a0*b0**2+d)/2.
    sigma_s[i] = invgamma.rvs(a, scale = b0)

In [161]:
i = 0
C0 = 1e2
aux1= zz[:,:(i+1)].T@zz[:,:(i+1)]
inv_C = C0**(-1)*np.eye(K) + sigma[i]**(-2)*aux1
C = inv(inv_C)
C

array([[ 50.01468412, -49.98531588],
       [-49.98531588,  50.01468412]])

In [162]:
zz[:,:(i+1)].T.shape

(1, 10)

In [163]:
aux2= zz[:,:(i+1)].T@yy[:,i]
aux2
# C@(C0**(-1)*np.ones(i+1) + 

array([19.21833021])

In [164]:
C@(C0**(-1)*np.ones(i+1) +sigma[i]**(-2)*aux2 )

ValueError: shapes (2,2) and (1,) not aligned: 2 (dim 1) != 1 (dim 0)